___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4



#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange



1. Підключитися до API.


In [3]:
import requests
site = 'https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?valcode=USD&date=20210811&json'

r = requests.get(site)
r

<Response [200]>

2. Отримати курс долара за останній рік.

In [4]:
from statistics import mean
from datetime import date, datetime, timedelta

def connect_to_api(date):
    param = {'date': date.strftime('%Y%m%d'), 'valcode': 'USD' }
    r = requests.get('https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?json', params=param)
    if r.status_code == 200:
        return r.json()[0]['rate']
    return None


def get_dollar_year():
    DB = {}
    start_date = date.today() - timedelta(days=365) 
    current_date = date.today()
    while start_date <= current_date:
        rate = connect_to_api(start_date)
        DB[start_date] = rate
        start_date += timedelta(days=1)
    return DB

db = get_dollar_year()
# print(db)
db_mean = mean(list(db.values()))
print(f'Середній курс долара за останній рік  {db_mean}')

Середній курс долара за останній рік  27.814918306010927


3. Вивести середнє значення та відхилення курсу за кожний місяць.

In [5]:
year_ago = date.today() - timedelta(days=365) 

rate_list = [(
    connect_to_api(year_ago + timedelta(days=i)), # курс долара
    (year_ago + timedelta(days=i)).year * 100 +   # дата (рік місяць)
    (year_ago + timedelta(days=i)).month   
    )
    for i in range (365)]


In [6]:
# унікальні значення дати (рік місяць)
year_month = set(map(lambda i: i[1], rate_list))

In [7]:
from statistics import mean, stdev

res = [] 
for j in year_month:
    month = str(j)[-2:]
    year = str(j)[:4]
    data = list(i[0] for i in rate_list if i[1]==j)
    mean_value = mean(data)
    std_dev = stdev(data)
    res.append((month, year, mean_value, std_dev))
    
res = sorted(res, key = lambda k: (k[1],k[0]))
res

[('08', '2020', 27.412911764705882, 0.07505146469550882),
 ('09', '2020', 27.97671666666667, 0.2409137677795668),
 ('10', '2020', 28.324951612903227, 0.055517948274816174),
 ('11', '2020', 28.310153333333332, 0.1508870820853458),
 ('12', '2020', 28.16971935483871, 0.24976838393380232),
 ('01', '2021', 28.219770967741937, 0.1231532113359842),
 ('02', '2021', 27.884660714285715, 0.10631085913415092),
 ('03', '2021', 27.79555806451613, 0.11551466219833594),
 ('04', '2021', 27.930006666666667, 0.09083219820022562),
 ('05', '2021', 27.602848387096774, 0.12045124565833751),
 ('06', '2021', 27.239873333333332, 0.15608484857289326),
 ('07', '2021', 27.214548387096773, 0.1558277165568175),
 ('08', '2021', 26.846407142857142, 0.06610510819649482)]

4. Дану інформацію записати у файл за допомогою pickle.

In [8]:
# записати res у файл
import pickle

f = open('year_rate_by_month.pickle', 'wb')
pickle.dump(res, f)

f.close()

In [9]:
# завантажити дані з файлу 
import pickle

f = open('year_rate_by_month.pickle', 'rb')
new_res = pickle.load(f)
f.close()

In [10]:
res

[('08', '2020', 27.412911764705882, 0.07505146469550882),
 ('09', '2020', 27.97671666666667, 0.2409137677795668),
 ('10', '2020', 28.324951612903227, 0.055517948274816174),
 ('11', '2020', 28.310153333333332, 0.1508870820853458),
 ('12', '2020', 28.16971935483871, 0.24976838393380232),
 ('01', '2021', 28.219770967741937, 0.1231532113359842),
 ('02', '2021', 27.884660714285715, 0.10631085913415092),
 ('03', '2021', 27.79555806451613, 0.11551466219833594),
 ('04', '2021', 27.930006666666667, 0.09083219820022562),
 ('05', '2021', 27.602848387096774, 0.12045124565833751),
 ('06', '2021', 27.239873333333332, 0.15608484857289326),
 ('07', '2021', 27.214548387096773, 0.1558277165568175),
 ('08', '2021', 26.846407142857142, 0.06610510819649482)]

### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/shop/

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

In [11]:
import requests
import sqlite3
from bs4 import BeautifulSoup

HEADERS = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'ua-UA,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }

site = 'https://smallpacking.agrosem.ua/shop/'

def find_page_links(site):
    """Функція, яка знаходить всі посилання на можливі сторінки сайту"""
    pages_links = [site]
    while True:
        r = requests.get(site, headers=HEADERS)
        soup = BeautifulSoup(r.text, 'lxml')
        try:
            next_page = soup.find('a', class_='next page-numbers').get('href')
            pages_links.append(next_page)
            site = next_page
        except:
            print('Next page is not found')
            break
    return pages_links

def get_all_product_links(site, db):
    """Функція, яка знаходить всі посилання товарів на site"""
    r = requests.get(site, headers=HEADERS)
    soup = BeautifulSoup(r.text, 'lxml')
    
    for teg in soup.findAll('div', class_='product-from-category-container'):
        link = teg.a.get('href').strip('/')
        db.append(link)

def get_info(site, product_list):
    """Функція, яка добавляє записи в словник 
    товару (sku, назва товару, ціна, тип упаковки)"""
    r = requests.get(site, headers=HEADERS)
    soup = BeautifulSoup(r.text, 'lxml')  

    sku = soup.find('span',itemprop='productID').text[5:]
    name = soup.find('div', class_="atributes-description").h2.text
    price = soup.find('div', class_='price-container').span.text
    price = int(price.replace(" ", "")[:-3])
    pack_type = soup.find('span', class_="type-packaging-name").text
    temp = [sku, name, price, pack_type]
    product_list.append(temp)     
    

def create_and_fill_database(name, list_values):
    # Створити базу даних
    conn = sqlite3.connect(name + '.sqlite3')
    curs = conn.cursor()
    curs.execute("CREATE TABLE Product (sku TEXT, name TEXT, price INTEGER, pack_type TEXT)")

    # Додати дані в таблицю
    for i in range(len(list_values)):
        curs.execute("INSERT INTO Product (sku, name, price, pack_type)"
                     " VALUES (?, ?, ?, ?)",
                     (list_values[i][0], list_values[i][1], list_values[i][2], list_values[i][3]))
    conn.commit()
    conn.close()

if __name__ == '__main__':
    all_product_links = []
    product_list = []

    pages_links = find_page_links(site)

    for page in pages_links:
        get_all_product_links(page, all_product_links)

    for link in all_product_links:
        get_info(link, product_list)
        
    create_and_fill_database('products',product_list)
    

Next page is not found


Простий запит на вивід усіх даних з таблиці Product

In [12]:
conn = sqlite3.connect('products.sqlite3')
curs = conn.cursor()
curs.execute('SELECT * FROM Product ORDER BY name')
rows = curs.fetchall()
 
for row in rows:
    print("{0} {1} {2} {3}".format(row[0], row[1], row[2], row[3]))
conn.close()


28164 Ferticare Hydro 6-14-30 + micro 1900 Коробка
66353 Ferticare Kombi 14-11-25 + micro 1700 Коробка
39566 MagPlon K (нітрат калію) 1200 Коробка
69156 Pulan 34.4 7500 Коробка
74266 Saletrosan®26 4400 Коробка
26252 Saletrosan®30 6000 Коробка
67325 Saletrzak 27 macro 4300 Коробка
76913 Siarczan AS 21 Macro 4700 Коробка
51667 Yara Liva Кальциніт 550 Коробка
95975 Yara Liva Нітрабор 550 Коробка
93119 YaraBela Сульфан 9500 Коробка
35951 YaraIrix 23000 Коробка
21521 YaraMila Complex 12-11-18 850 Коробка
31357 YaraMila Cropcare 11-11-21 850 Коробка
47993 YaraMila NP 20-20 +9,0SO3 12600 Коробка
89147 YaraMila NPK 12-24-12 14700 Коробка
84721 YaraMila NPK 16-16-16 12200 Коробка
63243 YaraMila NPK 16-27-7 15600 Коробка
24963 YaraMila NPK 18-11-13 11300 Коробка
24344 YaraMila NPK 7-20-28 15600 Коробка
64776 YaraMila NPK 8-24-24 15800 Коробка
66872 YaraMila NPK 9-12-25 + 2MgO + 6,5SO3 + micro 12800 Коробка
19571 YaraTera SOP (сульфат калію) 1100 Коробка
16332 YaraVita Rexolin ABC (комплекс мікро

### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

1. Виведіть інформацію про дану базу.

In [13]:
sqlite3.version

'2.6.0'

In [14]:
sqlite3.sqlite_version

'3.35.4'

2. Виведіть перелік всіх таблиць.

In [30]:
query = """
        SELECT name 
        FROM sqlite_master 
        WHERE type='table' 
        ORDER BY Name
        """
con = sqlite3.connect('tysql.sqlite')
cursor = con.cursor()
cursor.execute(query)
tables = list(map(lambda t: t[0], cursor.fetchall()))
print(tables)

['Customers', 'OrderItems', 'Orders', 'Products', 'Vendors']


3. Список всіх cust_id з таблиці Customers table.

In [31]:
query = """
        SELECT cust_id 
        FROM Customers
        """
cursor.execute(query)
print(cursor.fetchall())

[('1000000001',), ('1000000002',), ('1000000003',), ('1000000004',), ('1000000005',)]


4. Всю таблицю Customers table

In [32]:
query = """
        SELECT * 
        FROM Customers
        """
cursor.execute(query)
result = cursor.fetchall()
for row in result:
    print(row, '\n')

('1000000001', 'Village Toys', '200 Maple Lane', 'Detroit', 'MI', '44444', 'USA', 'John Smith', 'sales@villagetoys.com') 

('1000000002', 'Kids Place', '333 South Lake Drive', 'Columbus', 'OH', '43333', 'USA', 'Michelle Green', None) 

('1000000003', 'Fun4All', '1 Sunny Place', 'Muncie', 'IN', '42222', 'USA', 'Jim Jones', 'jjones@fun4all.com') 

('1000000004', 'Fun4All', '829 Riverside Drive', 'Phoenix', 'AZ', '88888', 'USA', 'Denise L. Stephens', 'dstephens@fun4all.com') 

('1000000005', 'The Toy Store', '4545 53rd Street', 'Chicago', 'IL', '54545', 'USA', 'Kim Howard', None) 



5. Список клієнтів (cust_names) відсортованих від Z до A

In [33]:
query = """
        SELECT cust_name
        FROM Customers
        ORDER BY cust_name DESC
        """
cursor.execute(query)
result = cursor.fetchall()
for row in result:
    print(row, '\n')

('Village Toys',) 

('The Toy Store',) 

('Kids Place',) 

('Fun4All',) 

('Fun4All',) 



6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

In [34]:
query = """
        SELECT *
        FROM Orders JOIN Customers ON Orders.cust_id = Customers.cust_id
        ORDER BY  cust_name ASC, order_date 
        """
cursor.execute(query)
result = cursor.fetchall()
for row in result:
    print(row, '\n')

(20006, '2012-01-12', '1000000003', '1000000003', 'Fun4All', '1 Sunny Place', 'Muncie', 'IN', '42222', 'USA', 'Jim Jones', 'jjones@fun4all.com') 

(20007, '2012-01-30', '1000000004', '1000000004', 'Fun4All', '829 Riverside Drive', 'Phoenix', 'AZ', '88888', 'USA', 'Denise L. Stephens', 'dstephens@fun4all.com') 

(20008, '2012-02-03', '1000000005', '1000000005', 'The Toy Store', '4545 53rd Street', 'Chicago', 'IL', '54545', 'USA', 'Kim Howard', None) 

(20009, '2012-02-08', '1000000001', '1000000001', 'Village Toys', '200 Maple Lane', 'Detroit', 'MI', '44444', 'USA', 'John Smith', 'sales@villagetoys.com') 

(20005, '2012-05-01', '1000000001', '1000000001', 'Village Toys', '200 Maple Lane', 'Detroit', 'MI', '44444', 'USA', 'John Smith', 'sales@villagetoys.com') 



7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

In [35]:
query = """
        SELECT prod_id, quantity, item_price
        FROM OrderItems
        ORDER BY quantity DESC, item_price DESC
        """
cursor.execute(query)
result = cursor.fetchall()
for row in result:
    print(row, '\n')

('BNBG01', 250, 2.49) 

('BNBG02', 250, 2.49) 

('BNBG03', 250, 2.49) 

('BR03', 100, 10.99) 

('BR01', 100, 5.49) 

('BNBG01', 100, 2.99) 

('BNBG02', 100, 2.99) 

('BNBG03', 100, 2.99) 

('BR03', 50, 11.49) 

('RGAN01', 50, 4.49) 

('BR01', 20, 5.99) 

('BR03', 10, 11.99) 

('BR02', 10, 8.99) 

('BNBG01', 10, 3.49) 

('BNBG02', 10, 3.49) 

('BNBG03', 10, 3.49) 

('BR03', 5, 11.99) 

('RGAN01', 5, 4.99) 



8. Товар (з таблиці Products), ціна якого становить 9.49

In [36]:
query = """
        SELECT *
        FROM Products
        WHERE prod_price == 9.49
        """
cursor.execute(query)
result = cursor.fetchall()
for row in result:
    print(row, '\n')

('RYL01', 'FNG01', 'King doll', 9.49, '12 inch king doll with royal garments and crown') 

('RYL02', 'FNG01', 'Queen doll', 9.49, '12 inch queen doll with royal garments and crown') 



9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

In [37]:
query = """
        SELECT prod_name, prod_price
        FROM Products
        WHERE prod_price BETWEEN 3 AND 6
        ORDER BY prod_price ASC
        """
cursor.execute(query)
result = cursor.fetchall()
for row in result:
    print(row, '\n')

('Fish bean bag toy', 3.49) 

('Bird bean bag toy', 3.49) 

('Rabbit bean bag toy', 3.49) 

('Raggedy Ann', 4.99) 

('8 inch teddy bear', 5.99) 



10. Кількість товару, що було продано

In [38]:
query = """
        SELECT SUM(quantity) as selled_products
        FROM OrderItems
        """
cursor.execute(query)
result = cursor.fetchall()
for row in result:
    print(row, '\n')

(1430,) 



11. Кількість найменувань товару, ціна якого більша за 4

In [39]:
query = """
        SELECT prod_name 
        FROM Products
        WHERE prod_price > 4
        """
cursor.execute(query)
result = cursor.fetchall()
for row in result:
    print(row, '\n')

('8 inch teddy bear',) 

('12 inch teddy bear',) 

('18 inch teddy bear',) 

('Raggedy Ann',) 

('King doll',) 

('Queen doll',) 



12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

In [40]:
query = """
        SELECT prod_name, prod_price
        FROM Products
        ORDER BY prod_price DESC
        LIMIT 3
        """
cursor.execute(query)
result = cursor.fetchall()
for row in result:
    print(row, '\n')

('18 inch teddy bear', 11.99) 

('King doll', 9.49) 

('Queen doll', 9.49) 



13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

In [41]:
query = """
        SELECT cust_id, cust_name, COUNT(order_num)
        FROM (SELECT Customers.cust_id, Orders.order_num, cust_name
              FROM Customers JOIN Orders ON Customers.cust_id=Orders.cust_id) as t
        GROUP BY cust_id
        """
cursor.execute(query)
result = cursor.fetchall()
for row in result:
    print(row, '\n')


('1000000001', 'Village Toys', 2) 

('1000000003', 'Fun4All', 1) 

('1000000004', 'Fun4All', 1) 

('1000000005', 'The Toy Store', 1) 



13. Підрахуйте кількість замовлень для кожного клієнта використовуючи GROUP BY

In [42]:
query = """
        SELECT Customers.cust_id, cust_name, COUNT(*) as count_orders
        FROM Orders JOIN Customers ON Orders.cust_id = Customers.cust_id
        GROUP BY Customers.cust_id
        """
cursor.execute(query)
result = cursor.fetchall()
for row in result:
    print(row, '\n')

('1000000001', 'Village Toys', 2) 

('1000000003', 'Fun4All', 1) 

('1000000004', 'Fun4All', 1) 

('1000000005', 'The Toy Store', 1) 



14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

In [43]:
query = """
        SELECT Customers.cust_id, cust_name, order_num, order_date
        FROM Orders JOIN Customers ON Orders.cust_id = Customers.cust_id
        """
cursor.execute(query)
result = cursor.fetchall()
for row in result:
    print(row, '\n')

('1000000001', 'Village Toys', 20005, '2012-05-01') 

('1000000003', 'Fun4All', 20006, '2012-01-12') 

('1000000004', 'Fun4All', 20007, '2012-01-30') 

('1000000005', 'The Toy Store', 20008, '2012-02-03') 

('1000000001', 'Village Toys', 20009, '2012-02-08') 



15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

In [46]:
query = """
        SELECT prod_price, prod_name
        FROM Products
        WHERE prod_price < 5
        
        UNION 
        
        SELECT prod_price, prod_name
        FROM Products
        WHERE prod_price >= 5

        
        """
cursor.execute(query)
result = cursor.fetchall()
for row in result:
    print(row, '\n')
con.close()

(3.49, 'Bird bean bag toy') 

(3.49, 'Fish bean bag toy') 

(3.49, 'Rabbit bean bag toy') 

(4.99, 'Raggedy Ann') 

(5.99, '8 inch teddy bear') 

(8.99, '12 inch teddy bear') 

(9.49, 'King doll') 

(9.49, 'Queen doll') 

(11.99, '18 inch teddy bear') 



### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди


**1.**

```mysql
SELECT model, speed, hd
FROM PC
WHERE price < 500
```

**2.**

```mysql
SELECT DISTINCT maker
FROM Product
WHERE type = 'Printer'
```

**3.**

```mysql
SELECT model, ram, screen
FROM Laptop
WHERE price > 1000
```

**4.**

```mysql
SELECT *
FROM Printer
WHERE color = 'y'
```

**5.**

```mysql
SELECT model, speed, hd
FROM PC
WHERE price < 600 AND cd IN ('12x', '24x')
```

**6.**

```mysql
SELECT DISTINCT maker, speed 
FROM Product JOIN Laptop ON Product.model = Laptop.model 
WHERE Laptop.hd >= 10 
ORDER BY maker
```

**7.1**

```mysql
SELECT PC.model, price
FROM Product JOIN PC ON Product.model=PC.model 
WHERE maker = 'B'

UNION

SELECT Laptop.model, price
FROM Product JOIN Laptop ON Product.model=Laptop.model 
WHERE maker = 'B'

UNION 

SELECT Printer.model, price
FROM Product JOIN Printer ON Product.model=Printer.model 
WHERE maker = 'B'
```

**7.2**

```mysql
SELECT PC.model, price
FROM PC 
WHERE model IN (SELECT model FROM Product WHERE maker = 'B')

UNION

SELECT Laptop.model, price
FROM Laptop
WHERE model IN (SELECT model FROM Product WHERE maker = 'B')

UNION 

SELECT Printer.model, price
FROM Printer
WHERE model IN (SELECT model FROM Product WHERE maker = 'B')
```

**8.**

```mysql
SELECT maker
FROM PRODUCT 
WHERE type = 'PC' 

EXCEPT

SELECT  maker
FROM PRODUCT 
WHERE type = 'Laptop'
```

**9.**

```mysql
SELECT DISTINCT maker
FROM Product JOIN PC ON Product.model = PC.model
WHERE speed >=450
```

**10.**

```mysql
SELECT model, price
FROM Printer
WHERE price IN (SELECT MAX(price)
                FROM Printer)
```

**11.**

```mysql
SELECT AVG(speed)
FROM PC

```

**12.**

```mysql
SELECT AVG(speed)
FROM Laptop
WHERE price > 1000
```

**13.**

```mysql
SELECT AVG(speed)
FROM PC JOIN Product ON PC.model=Product.model
WHERE maker = 'A'
```

**14.**

```mysql
SELECT Classes.class, name, country
FROM Classes JOIN Ships ON Classes.class = Ships.class
WHERE numGuns >= 10
```

**15.**

```mysql
SELECT hd
FROM PC 
GROUP BY hd
HAVING COUNT(*) >=2
```

**16.**

```mysql
SELECT DISTINCT A.model AS model, B.model AS model, A.speed As speed, A.ram As ram 
FROM PC AS A, PC B 
WHERE A.speed = B.speed AND A.ram = B.ram AND A.model > B.model
```

**17.**

```mysql
SELECT DISTINCT type as Type, Laptop.model as Model, speed
FROM Laptop JOIN Product ON Laptop.model=Product.model
WHERE speed < (SELECT MIN(speed) 
               FROM PC)
```


**18.**

```mysql
SELECT DISTINCT maker, price
FROM Product JOIN Printer ON Product.model=Printer.model
WHERE color = 'y' AND price = (SELECT MIN(price)
                               FROM Printer
                               WHERE color = 'y')
```

**19.**

```mysql
SELECT maker, AVG(screen)
FROM Product JOIN Laptop ON Product.model=Laptop.model
GROUP BY maker
```

**20.**

```mysql
SELECT maker, COUNT(model) AS Count_Model
FROM Product 
WHERE type='PC'
GROUP BY maker
HAVING COUNT(Model) >=3
```

**21.**

```mysql
SELECT maker, MAX(price)
FROM Product JOIN PC ON Product.model=PC.model
GROUP BY maker
```


**22.**

```mysql
SELECT speed, AVG(price)
FROM PC
WHERE speed > 600
GROUP BY speed
```

**23.**

```mysql
SELECT DISTINCT maker
FROM Product
WHERE maker IN (
	SELECT DISTINCT maker 
	FROM Product JOIN PC ON Product.model = PC.model
	WHERE speed >= 750
)
AND maker IN (
	SELECT DISTINCT maker 
	FROM Product JOIN Laptop ON Product.model = Laptop.model
    WHERE speed >= 750
)
```

**24.**

```mysql
WITH MAX
AS (SELECT model, price FROM PC
    UNION 
    SELECT model, price FROM Laptop
    UNION 
    SELECT model, price FROM Printer)
    
SELECT model
FROM MAX
WHERE price = (SELECT MAX(price)
               FROM MAX)
```

**25.**

```mysql
SELECT DISTINCT maker
FROM Product JOIN PC ON Product.model = PC.model
WHERE ram = (SELECT MIN(ram)
             FROM PC)
AND speed = (SELECT MAX(speed)
             FROM PC
             WHERE ram = (SELECT MIN(ram)
		                  FROM PC))
AND maker IN (SELECT maker
	      FROM Product
	      WHERE type='Printer')
```

**26.**

```mysql
WITH Avg_price
AS
(
SELECT price 
FROM Product JOIN PC ON Product.model=PC.model
WHERE maker='A'

UNION ALL

SELECT price
FROM Product JOIN Laptop ON Product.model=Laptop.model
WHERE maker='A'
)

SELECT AVG(price) as AVG_price
FROM Avg_price
```


**27.**

```mysql
SELECT maker, AVG(hd) AS Avg_hd
FROM Product JOIN PC ON Product.model=PC.model
WHERE maker IN (SELECT maker
                FROM Product
                WHERE type = 'Printer')
GROUP BY maker

```

**28.**

```mysql
SELECT COUNT(maker) as qty
FROM (SELECT maker
      FROM Product
      GROUP BY maker
      HAVING COUNT(*) = 1) as T
```

**29.**

```mysql
SELECT i.point, i.date, i.inc, o.out
FROM Income_o i LEFT JOIN Outcome_o o ON i.point = o.point AND i.date = o.date

UNION

SELECT o.point, o.date, i.inc, o.out
FROM Outcome_o o LEFT JOIN Income_o i ON o.point = i.point AND o.date = i.date
```

**30.**

```mysql
SELECT point, date, SUM(sum_out), SUM(sum_inc)
FROM (
SELECT point, date, SUM(inc) as sum_inc, null as sum_out
FROM  Income
GROUP BY point, date

UNION

SELECT point, date, null as sum_inc, SUM(out) as sum_out
FROM  Outcome
GROUP BY point, date
) as T
GROUP BY point, date
ORDER BY point
```
